## Import Packages

In [1]:
from tdw.controller import Controller
from tdw.add_ons.oculus_touch import OculusTouch
from tdw.vr_data.oculus_touch_button import OculusTouchButton
from tdw.output_data import OutputData
from tdw.add_ons.third_person_camera import ThirdPersonCamera
from tdw.tdw_utils import TDWUtils
from tdw.librarian import ModelLibrarian
from tdw.output_data import OutputData, Bounds
from scipy.spatial.transform import Rotation as R
from tdw.add_ons.image_capture import ImageCapture
from tdw.quaternion_utils import QuaternionUtils
from tdw.librarian import ModelLibrarian
from tdw.add_ons.mouse import Mouse
from tdw.add_ons.keyboard import Keyboard
import numpy as np
import os
import time

## Create Libraries/Folder to save recorded files
IMPORTANT: If you are using Mac, replace "Windows" with "Darwin" in the 6th line and if you are using Linux, replace it with "Linux"

In [2]:
current_dir = os.path.dirname(os.getcwd())
if os.path.exists(os.path.join(current_dir, "assets")):
    local_librarian = ModelLibrarian(
        library=os.path.join(current_dir, "assets", "library.json")
    )
    models_directory = (
        os.path.join(current_dir, "assets", "Windows").replace("\\", "/") + "/"
    )
else:
    print("No local models found")
    models_directory = None
    local_librarian = None

images_dir = os.path.join(current_dir, "images")
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

videos_dir = os.path.join(current_dir, "videos")
if not os.path.exists(videos_dir):
    os.makedirs(videos_dir)

No local models found


## Helper functions

In [3]:
def get_bounds(c, obj_id, rotation):
    resp = c.communicate({"$type": "send_bounds", "frequency": "once"})
    for i in range(len(resp) - 1):
        r_id = OutputData.get_data_type_id(resp[i])
        if r_id == "boun":
            bounds = Bounds(resp[i])
            for j in range(bounds.get_num()):
                if bounds.get_id(j) == obj_id:
                    top = bounds.get_top(j)[1]
                    bottom = bounds.get_bottom(j)[1]
                    back = bounds.get_back(j)[2]
                    front = bounds.get_front(j)[2]
                    left = bounds.get_left(j)[0]
                    right = bounds.get_right(j)[0]
    init_corners = np.array(
        [
            [left, top, back],
            [left, top, front],
            [left, bottom, back],
            [left, bottom, front],
            [right, top, back],
            [right, top, front],
            [right, bottom, back],
            [right, bottom, front],
        ]
    )

    r = (
        R.from_euler(
            "xyz", [[rotation["x"], rotation["y"], rotation["z"]]], degrees=True
        )
        .as_matrix()
        .squeeze()
    )

    transformed_corners = np.matmul(r, init_corners.T).T

    return {
        "left": float(left),
        "right": float(right),
        "top": float(top),
        "bottom": float(bottom),
        "front": float(front),
        "back": float(back),
        "width": abs(float(left - right)),
        "height": abs(float(top - bottom)),
        "depth": abs(float(front - back)),
        "corners": transformed_corners.tolist(),
    }


def add_your_own_object(
    c,
    name,
    id,
    position,
    rotation={"x": 0, "y": 0, "z": 0},
    mass=5,
    color=None,
    scale={"x": 1, "y": 1, "z": 1},
    material=None,
    dynamic_friction=0.3,
    static_friction=0.3,
    bounciness=0.7,
    set_kinematic=False,
    use_gravity=True
):
    commands = []
    commands.extend(
        [
            {
                "$type": "add_object",
                "name": name,
                "url": "file:///" + models_directory + name,
                "id": id,
                "position": position,
            },
            {
                "$type": "scale_object",
                "scale_factor": scale,
                "id": id,
            },
        ]
    )
    if material:
        model_record = local_librarian.get_record(name)
        commands.extend(
            TDWUtils.set_visual_material(
                c=c,
                substructure=model_record.substructure,
                material=material,
                object_id=id,
            )
        )
    commands.append({"$type": "set_mass", "mass": mass, "id": id})
    commands.extend(
        [
            {
                "$type": "rotate_object_by",
                "angle": rotation["x"],
                "id": id,
                "axis": "pitch",
                "use_centroid": True,
            },
            {
                "$type": "rotate_object_by",
                "angle": rotation["y"],
                "id": id,
                "axis": "yaw",
                "use_centroid": True,
            },
            {
                "$type": "rotate_object_by",
                "angle": rotation["z"],
                "id": id,
                "axis": "roll",
                "use_centroid": True,
            },
        ]
    )

    commands.extend(
        [
            {
                "$type": "set_kinematic_state",
                "id": id, 
                "is_kinematic": set_kinematic, 
                "use_gravity": use_gravity
            },
            {
                "$type": "set_physic_material",
                "dynamic_friction": dynamic_friction, 
                "static_friction": static_friction, 
                "bounciness": bounciness, 
                "id": id
            }
        ]
    )

    if color:
        commands.append({"$type": "set_color", "color": color, "id": id})
    c.communicate(commands)
    bounds = get_bounds(c, id, rotation)
    c.communicate([])

    return bounds

def add_tdw_object(
    c,
    name,
    id,
    position,
    rotation={"x": 0, "y": 0, "z": 0},
    mass=5,
    color=None,
    scale={"x": 1, "y": 1, "z": 1},
    material=None,
    dynamic_friction=0.3,
    static_friction=0.3,
    bounciness=0.7,
    set_kinematic=False,
    use_gravity=True,
    library="models_core.json"
):
    commands = []
    commands.extend(
        c.get_add_physics_object(
            model_name=name,
            object_id=id,
            position=position,
            rotation=rotation,
            mass=mass,
            default_physics_values = False,
            dynamic_friction=dynamic_friction,
            static_friction=static_friction,
            bounciness=bounciness,
            gravity=use_gravity,
            kinematic=set_kinematic,
            scale_factor=scale,
            library=library
        )
    )

    if color:
        commands.append({"$type": "set_color", "color": color, "id": id})

    if material:
        librarian = ModelLibrarian(library=library)
        model_record = librarian.get_record(name)
        commands.extend(
            TDWUtils.set_visual_material(
                c=c,
                substructure=model_record.substructure,
                material=material,
                object_id=id,
            )
        )

    c.communicate(commands)
    bounds = get_bounds(c, id, rotation)
    
    return bounds

## Scene setup functions

This is basically the same code you wrote in setup_a_simple_scene. This cell may take a few minutes to run...

In [4]:
def setup_scene(c, is_vr=False):
    # c.communicate(c.get_add_scene(scene_name="tdw_room"))
    c.communicate(
        [
            TDWUtils.create_empty_room(15, 15),
            {"$type": "set_screen_size", "width": 1000, "height": 1000},
            {"$type": "set_target_framerate", "framerate": 40},
        ]
    )
    camera_position = {"x": 0, "y": 1.6, "z": -1.0}
    if is_vr == False:
        camera_1 = ThirdPersonCamera(avatar_id="a",
                                position=camera_position,
                                rotation={"x": 0, "y": 0, "z": 0},
                                field_of_view=105
        )
        c.add_ons.append(camera_1)
    table_stats = {
        "name" : "small_table_green_marble",
        "id" : c.get_unique_id(),
        "position" : {"x" : 0.0, "y" : 0.0, "z" : 0},
        "rotation" : {"x" : 0, "y" : 0, "z" : 0},
        "scale" : {"x" : 1.5, "y" : 1.2, "z" : 1.5},
        "color" : None,
        "material" : None,
        "mass" : 2000,
        "dynamic_friction" : 0.45,
        "static_friction" : 0.48,
        "bounciness" : 0.0,
        "set_kinematic" : False,
        "use_gravity" : True
    }
    table_bounds = add_tdw_object(c, **table_stats)

    station_stats = {
        "name" : "iron_box",
        "id" : c.get_unique_id(),
        "position" : {"x" : 0, "y" : table_bounds["top"], "z" : 0.15},
        "rotation" : {"x" : 0, "y" : 0, "z" : 0},
        "scale" : {"x" : 2.0, "y" : 0.7, "z" : 4.0},
        "color" : None,
        "material" : "parquet_wood_red_cedar",
        "mass" : 1000,
        "dynamic_friction" : 0.15,
        "static_friction" : 0.18,
        "bounciness" : 0.0,
        "set_kinematic" : False,
        "use_gravity" : True,
    }
    station_bounds = add_tdw_object(c, **station_stats)

    ramp_stats = {
        "name" : "ramp_with_platform_weld",
        "id" : c.get_unique_id(),
        "position" : {"x" : 0, "y" : station_bounds["top"], "z" : 0.15},
        "rotation" : {"x" : 0, "y" : -90, "z" : 0},
        "scale" : {"x" : 0.2, "y" : 0.4, "z" : 0.4},
        "color" : {"r" : 0.7, "g" : 0.7, "b" : 1},
        "mass" : 1000,
        "dynamic_friction" : 0.15,
        "static_friction" : 0.18,
        "bounciness" : 0.8,
        "set_kinematic" : False,
        "use_gravity" : True,
        "library" : "models_full.json"
    }
    ramp_bounds = add_tdw_object(c, **ramp_stats)

    left_wall_stats = {
        "name" : "iron_box",
        "id" : c.get_unique_id(),
        "position" : {"x" : -0.42, "y" : table_bounds["top"], "z" : 0.05},
        "rotation" : {"x" : 0, "y" : 0, "z" : 0},
        "scale" : {"x" : 0.1, "y" : 2.5, "z" : 3.0},
        "color" : None,
        "material" : "parquet_wood_red_cedar",
        "mass" : 1000,
        "dynamic_friction" : 0.15,
        "static_friction" : 0.18,
        "bounciness" : 0.8,
        "set_kinematic" : False,
        "use_gravity" : True
    }
    left_wall_bounds = add_tdw_object(c, **left_wall_stats)

    right_wall_stats = {
        "name" : "iron_box",
        "id" : c.get_unique_id(),
        "position" : {"x" : 0.42, "y" : table_bounds["top"], "z" : 0.05},
        "rotation" : {"x" : 0, "y" : 0, "z" : 0},
        "scale" : {"x" : 0.1, "y" : 2.5, "z" : 3.0},
        "color" : None,
        "material" : "parquet_wood_red_cedar",
        "mass" : 1000,
        "dynamic_friction" : 0.15,
        "static_friction" : 0.18,
        "bounciness" : 0.8,
        "set_kinematic" : False,
        "use_gravity" : True
    }
    right_wall_bounds = add_tdw_object(c, **right_wall_stats)

    ball_stats = {
        "name" : "prim_sphere",
        "id" : c.get_unique_id(),
        "position" : {"x" : 0, "y" : ramp_bounds["top"] + 0.1, "z" : 0.25},
        "rotation" : {"x" : 0, "y" : 0, "z" : 0},
        "scale" : {"x" : 0.06, "y" : 0.06, "z" : 0.06},
        "color" : {"r" : 0.8, "g" : 0, "b" : 0},
        "mass" : 2.0,
        "dynamic_friction" : 0.15,
        "static_friction" : 0.18,
        "bounciness" : 0.8,
        "set_kinematic" : False,
        "use_gravity" : True,
        "library" : "models_special.json"
    }
    ball_bounds = add_tdw_object(c, **ball_stats)
    all_ids = {
        "table" : table_stats["id"],
        "station" : station_stats["id"],
        "ramp" : ramp_stats["id"],
        "left_wall" : left_wall_stats["id"],
        "right_wall" : right_wall_stats["id"],
        "ball" : ball_stats["id"]
    }
    return all_ids, camera_position, table_bounds["top"]

## An example of scene setup

In [5]:
c = Controller(port = np.random.randint(1, 10000))
all_ids, camera_position, table_height = setup_scene(c)
input("Press Enter to start the physics simulation")
c.communicate({"$type": "terminate"})

## Run experiment on monitor using mouse

In [6]:
def calculate_projection(mouse_pos, camera_position, table_height):
    # Calculate the projection of the mouse position on the table
    m = (mouse_pos[1] - camera_position["y"]) / (mouse_pos[0] - camera_position["x"])
    b = camera_position["y"] - m * camera_position["x"]
    x = (table_height - b) / m
    z = mouse_pos[2]
    return [x, table_height, z]

In [7]:
class Track_w_Mouse(Controller):
    def __init__(self):
        super().__init__(port = np.random.randint(1, 10000))
        # Initialize parameters and add mouese and keyboard
        self.mouse = Mouse(avatar_id="a")
        self.add_ons.append(self.mouse)
        self.keyboard = Keyboard()
        self.add_ons.append(self.keyboard)
        self.done = False
        self.force_applied = False

    def run(self, all_ids, camera_position, table_height):
        self.ball_id = all_ids["ball"]
        # add a bowl on the table at z = -0.4
        bowl_stats = {
            "name": "round_bowl_small_walnut",
            "id": self.get_unique_id(),
            "position": {"x": 0, "y": table_height, "z": -0.4},
            "rotation": {"x": 0, "y": 0, "z": 0},
            "scale": {"x": 0.3, "y": 0.3, "z": 0.3},
        }
        bowl_bounds = add_tdw_object(self, **bowl_stats)
        capture = ImageCapture(path=images_dir, avatar_ids=["a"])
        c.add_ons.append(capture)

        #turn of the physics when the trial begins
        self.communicate({"$type": "simulate_physics", "value": False})

        touched_positions = [] #create a list to save the positions which the bowl was moved to
        counter = 0 #set a counter for the frame number (updates only in frames after the force is applied)
        while counter < 50:
            self.communicate([])
            mouse_pos = self.mouse.world_position # save the position of the mouse in the world frame. Note that this is the end-point of the ray cast from the mouse
            projection_point = calculate_projection(mouse_pos, camera_position, table_height) # We want to know where the mouse is pointing on the table.
            # Move the bowl to the projection point
            self.communicate(
                [
                    {
                        "$type": "teleport_object",
                        "id": bowl_stats["id"],
                        "position": {"x": projection_point[0], "y": projection_point[1], "z": projection_point[2]},
                    },
                    {
                        "$type": "rotate_object_to",
                        "rotation": {"w": 1, "x": 0, "y": 0, "z": 0},
                        "id": bowl_stats["id"],
                    }
                ]
            )
            # Listen to the space key to apply a force to the ball
            self.keyboard.listen(key="Space", function=self.apply_force)
            if self.force_applied:
                # after the force is applied, we want to save the position of the bowl and update the counter
                touched_positions.append(projection_point)
                counter += 1

        return touched_positions, self.force_vector

    def apply_force(self):
        self.communicate({"$type": "simulate_physics", "value": True})
        force_vector = {"x": np.random.uniform(-0.002, 0.002), "y": 0, "z": np.random.uniform(-0.0005, 0)}
        print("force vector: {}".format(force_vector))
        self.communicate(
            {
                "$type": "apply_force_to_object",
                "id": self.ball_id,
                "force": force_vector,
            }
        )
        self.force_vector = force_vector
        self.force_applied = True

In [8]:
if __name__ == "__main__":
    c = Track_w_Mouse()
    all_ids, camera_position, table_height = setup_scene(c)
    mouse_positions, force_vector = c.run(all_ids, camera_position, table_height)
    c.communicate({"$type": "terminate"})

You are using TDW 1.12.18.0 but version 1.13.0.0 is available.
Consider upgrading:
pip3 install tdw -U
Build version 1.12.18
Unity Engine 2020.3.24f1
Python tdw module version 1.12.18.0
force vector: {'x': -0.0016723438983916733, 'y': 0, 'z': -0.0004829991317532105}


## Run experiment with VR

In [10]:
len(mouse_positions)

50

In [11]:
class VR_Experiment(Controller):
    def __init__(self):
        super().__init__(port = np.random.randint(1, 10000))
        self.done = False
        self.force_applied = False

    def run(self, all_ids, table_height):
        # Initialize the VR controller
        self.vr = OculusTouch(
            position={"x": 0, "y": 0, "z": -1.0}, non_graspable=[all_ids["table"]]
        )
        # Listen to the trigger button to apply a force to the ball
        self.vr.listen_to_button(
            button=OculusTouchButton.trigger_button,
            is_left=False,
            function=self.apply_force,
        )
        # Add the VR controller to the list of add-ons
        self.add_ons.extend([self.vr])
        # Show the loading screen
        self.vr.show_loading_screen(show=True)
        self.ball_id = all_ids["ball"]

        # Turn off the physics until the force is applied
        self.communicate({"$type": "simulate_physics", "value": False})
        self.vr.show_loading_screen(show=False)

        # Create lists to save the positions of the head and the right hand
        head = []
        right_hand = []
        s_trial_t = time.time()
        counter = 0
        while counter < 200:
            resp = self.communicate(
                {
                    "$type": "send_collisions",
                    "enter": True,
                    "stay": False,
                    "exit": False,
                    "collision_types": ["obj"],
                }
            )
            if self.force_applied:
                # Record the head and hand information after the force is applied
                head.append(
                    {
                        "position": TDWUtils.array_to_vector3(self.vr.head.position),
                        "rotation": TDWUtils.array_to_vector3(
                            QuaternionUtils.quaternion_to_euler_angles(
                                self.vr.head.rotation
                            )
                        ),
                        "time": time.time() - s_trial_t,
                    }
                )
                right_hand.append(
                    {
                        "position": TDWUtils.array_to_vector3(
                            self.vr.right_hand.position
                        ),
                        "rotation": TDWUtils.array_to_vector3(
                            QuaternionUtils.quaternion_to_euler_angles(
                                self.vr.right_hand.rotation
                            )
                        ),
                        "time": time.time() - s_trial_t,
                    }
                )
                counter += 1

        return head, right_hand, self.force_vector

    def apply_force(self):
        self.communicate({"$type": "simulate_physics", "value": True})
        force_vector = {"x": np.random.uniform(-0.002, 0.002), "y": 0, "z": np.random.uniform(-0.0005, 0)}
        print("force vector: {}".format(force_vector))
        c.communicate(
            {
                "$type": "apply_force_to_object",
                "id": self.ball_id,
                "force": force_vector,
            }
        )
        self.force_vector = force_vector
        self.force_applied = True

In [16]:
if __name__ == "__main__":
    c = VR_Experiment()
    all_ids, camera_position, table_height = setup_scene(c, is_vr=True)
    head, right_hand, force_vector = c.run(all_ids, table_height)
    c.communicate({"$type": "terminate"})

You are using TDW 1.12.18.0 but version 1.13.0.0 is available.
Consider upgrading:
pip3 install tdw -U
Build version 1.12.18
Unity Engine 2020.3.24f1
Python tdw module version 1.12.18.0
force vector: {'x': -0.0005353406167498331, 'y': 0, 'z': -0.00031381346282929195}
